In [1]:
import numpy as np
import pandas as pd

# Data Viz. 
import statsmodels.formula.api as smf
from statsmodels.tsa.seasonal import seasonal_decompose
from scipy.ndimage import gaussian_filter
from calendar import monthrange
from calendar import month_name

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
from matplotlib import rcParams
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
RSEED=42
import datetime, time, os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
import seaborn as sns
#from datetime import datetime

In [2]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

train data

In [3]:
df = pd.read_csv('data/train.csv')
df.columns

Index(['id', 'breath_id', 'R', 'C', 'time_step', 'u_in', 'u_out', 'pressure'], dtype='object')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6036000 entries, 0 to 6035999
Data columns (total 8 columns):
 #   Column     Dtype  
---  ------     -----  
 0   id         int64  
 1   breath_id  int64  
 2   R          int64  
 3   C          int64  
 4   time_step  float64
 5   u_in       float64
 6   u_out      int64  
 7   pressure   float64
dtypes: float64(3), int64(5)
memory usage: 368.4 MB


new variables from linear interpolation

In [5]:
more_var = pd.read_csv('data/utms.csv')
more_var.columns

Index(['id', 'R*C', '1/R*C', 'asc', 'mean', 'std', 'utm1', 'utm2', 'utm3'], dtype='object')

feature engineering

In [6]:
df['vol'] = df['time_step'] * df['u_in']
df['vol'] = df.query('u_out==0').groupby('breath_id')['vol'].cumsum()
df['rtime']=df['time_step'].apply(lambda x: round(x,3))
df.query('id%80==2')
df['step_id']=df.id.apply(lambda x: x%80)
df=df.query('u_out==0')
df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()
df['minus_one']=-1.0
df['plus_one']=1.0
df['exponent']=(df['minus_one']*df['time_step'])/(df['R']*df['C'])
df['factor']=np.exp(df['exponent'])
df['vf']=(df['u_in_cumsum']*df['R'])/df['factor']
df['vt']=0
df.loc[df['time_step'] != 0, 'vt']=df['vol']/(df['C']*(df['minus_one']*df['factor']+df['plus_one']))
df['v']=df['vf']+df['vt']

new full dataframe

In [7]:
new_df = df.merge(more_var, how='inner', on='id')
new_df.columns

Index(['id', 'breath_id', 'R', 'C', 'time_step', 'u_in', 'u_out', 'pressure',
       'vol', 'rtime', 'step_id', 'u_in_cumsum', 'minus_one', 'plus_one',
       'exponent', 'factor', 'vf', 'vt', 'v', 'R*C', '1/R*C', 'asc', 'mean',
       'std', 'utm1', 'utm2', 'utm3'],
      dtype='object')

In [8]:
cols=['v','u_in_cumsum','vol','pressure', 'u_in', 'R', 'C', 'time_step', 'step_id', 'R*C', '1/R*C', 'asc', 'mean',
       'std', 'utm1', 'utm2', 'utm3']
red_df=new_df[cols]
red_df

,v,u_in_cumsum,vol,pressure,u_in,R,C,time_step,step_id,R*C,1/R*C,asc,mean,std,utm1,utm2,utm3
0,1.666680,0.083334,0.000000,5.837492,0.083334,20,50,0.000000,1,1000,0.001,-1,22.119824,5.732006,0.083334,0.083334,0.083334
1,737.006954,18.466375,0.618632,5.907794,18.383041,20,50,0.033652,2,1000,0.001,-1,22.119824,5.732006,0.083334,18.383041,18.383041
2,1453.034477,40.975653,2.138333,7.876254,22.509278,20,50,0.067514,3,1000,0.001,-1,22.119824,5.732006,18.383041,0.083334,36.796840
3,2153.211108,63.784476,4.454391,11.742872,22.808822,20,50,0.101542,4,1000,0.001,-1,22.119824,5.732006,22.509278,18.383041,0.083334
4,2946.481672,89.140326,7.896588,12.234987,25.355850,20,50,0.135756,5,1000,0.001,-1,22.119824,5.732006,22.808822,22.509278,18.383041
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2290963,15962.475671,238.890288,66.643100,29.459013,1.869367,50,10,0.834147,26,500,0.002,1,8.168056,6.116885,2.650333,2.438287,3.783043
2290964,16025.086384,241.044703,68.512214,29.107502,2.154414,50,10,0.867574,27,500,0.002,1,8.168056,6.116885,1.869367,2.650333,2.438287
2290965,16010.376008,242.349137,69.687402,29.880825,1.304434,50,10,0.900917,28,500,0.002,1,8.168056,6.116885,2.154414,1.869367,2.650333
2290966,16046.588564,244.082966,71.307334,29.388710,1.733830,50,10,0.934309,29,500,0.002,1,8.168056,6.116885,1.304434,2.154414,1.869367


In [ ]:
red_df.describe()

In [9]:
#Variables for training
t_step="time_step"
full_data = red_df.drop(t_step, axis=1)
full_data.columns

Index(['v', 'u_in_cumsum', 'vol', 'pressure', 'u_in', 'R', 'C', 'step_id',
       'R*C', '1/R*C', 'asc', 'mean', 'std', 'utm1', 'utm2', 'utm3'],
      dtype='object')

In [10]:
#New dataframe with only training data - 5 columns
df_for_training = full_data.astype(float)


In [11]:
#LSTM uses sigmoid and tanh that are sensitive to magnitude so values need to be normalized
# normalize the dataset
scaler = StandardScaler()
scaler = scaler.fit(df_for_training)
df_for_training_scaled = scaler.transform(df_for_training)

In [12]:
#As required for LSTM networks, we require to reshape an input data into n_samples x timesteps x n_features. 
#In this example, the n_features is 16. We will make timesteps = 3 (past days data used for training). 

#Empty lists to be populated using formatted training data
trainX = []
trainY = []

n_future = 1   # Number of days we want to look into the future based on the past days.
n_past = 3  # Number of past days we want to use to predict the future.

#Reformat input data into a shape: (n_samples x timesteps x n_features)
#In my example, my df_for_training_scaled has a shape (1603677, 16)
#1603677 refers to the number of data points and 16 refers to the columns (multi-variables).
for i in range(n_past, len(df_for_training_scaled) - n_future +1):
    trainX.append(df_for_training_scaled[i - n_past:i, 0:df_for_training.shape[1]])
    trainY.append(df_for_training_scaled[i + n_future - 1:i + n_future, 0])

trainX, trainY = np.array(trainX), np.array(trainY)

print('trainX shape == {}.'.format(trainX.shape))
print('trainY shape == {}.'.format(trainY.shape))

#In my case, trainX has a shape (2290965, 3, 16). 
#2290965 because we are looking back 3 days (1603677 - 3 = 2290965). ?????
#Remember that we cannot look back 3 days until we get to the 4th day. 
#Also, trainY has a shape (2290965, 1). Our model only predicts a single value, but 
#it needs multiple variables (16 in my example) to make this prediction. 
#This is why we can only predict a single day after our training, the day after where our data ends.
#To predict more days in future, we need all the 16 variables which we do not have. 
#We need to predict all variables if we want to do that.

trainX shape == (2290965, 3, 16).
trainY shape == (2290965, 1).


In [13]:
# define the Autoencoder model

model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))
model.add(LSTM(32, activation='relu', return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(trainY.shape[1]))

model.compile(metrics='mae', optimizer=tf.keras.optimizers.Adam(0.001), loss='mae')
model.summary()

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-04-25 10:23:00.957659: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-04-25 10:23:00.959647: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 3, 64)             20736     
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 33,185
Trainable params: 33,185
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Define path for new directory 
root_logdir = os.path.join(os.curdir, "lstm_logs")
root_logdir

'./lstm_logs'

In [21]:
# Define function for creating a new folder for each run
def get_run_logdir():
    run_id = time.strftime('run_%d_%m_%Y-%H_%M_%S')
    return os.path.join(root_logdir, run_id)

In [22]:
# Create function for using callbacks; "name" should be the name of the model you use
def get_callbacks(name):
    return tf.keras.callbacks.TensorBoard(run_logdir+name, histogram_freq=1)

In [24]:
run_logdir = get_run_logdir()
run_logdir

'./lstm_logs/run_25_04_2022-10_26_09'

In [26]:
# fit the model
history = model.fit(trainX, trainY, epochs=2, batch_size=500, validation_split=0.2, verbose=1,
callbacks = get_callbacks("dnn_lstm"))

Epoch 1/2
 351/3666 [=>............................] - ETA: 3:25 - loss: 0.1204 - mae: 0.1204

KeyboardInterrupt: 

In [ ]:
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.legend()